In [2]:
# This is the FriendshipGoals CNN
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'1.13.2'

In [ ]:
# USING GOOGLE IMAGES FOR A DATASET

In [2]:
# Data Preprocessing
# need to apply transformations to test set to correct overfitting
# I don't know if Sample_Data is our actual test set tbh there's only 3 images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Dataset',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 317 images belonging to 2 classes.


In [3]:
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
# very similar to how in the ANN we used fit_transform on the training set, but just
# fit on the test set because we didn't want any information leakage from the training set.
# Don't want to mess with the images too much so they're intact like they would be in production
test_set = test_datagen.flow_from_directory('Dataset/Test_Data',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 0 images belonging to 0 classes.


In [ ]:
# building the model
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3, 
                               input_shape=[64,64,3], 
                               activation='relu'))
# filters = number of feature detectors
# input shape = (64, 64, 3) because we resized pictures to 64x64, and they're RGB
# kernel = size of feature detectors (e.g. 3 = 3x3 square)
# activation = activation function

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               kernel_size=3, 
                               activation='relu'))
# dont need input shape
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# flattening
cnn.add(tf.keras.layers.Flatten())

# full connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
# output layer
cnn.add(tf.keras.layers.Dense(units=3, activation='softmax')) # 3 classes